In [1]:
import os

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'train/' 'data/')
model_path = os.path.join(DATA_DIR, 'models/')

Prereqs:
- `conda install numpy scipy mkl-service libpython theano`
- `pip install keras= 1.0.7`
    - Why an older version of Keras? To avoid dwindling Theano support in Keras (now that Theano is deprecated). Issues with scipy on Windows? Try https://stackoverflow.com/a/39577864
- `conda install flask gevent requests pillow h5py`
- Switch to Theano as Keras backend: https://keras.io/backend/

In [2]:
from keras.layers import Embedding, LSTM, TimeDistributed
from keras.layers.core import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
# TODO: not as a const
chars = sorted(list('\x00\n\r !"&\'(),-./123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvwxyz}'))
char_indices = dict((c, i) for i, c in enumerate(chars))
vocab_size = len(chars)
n_fac = 42 # TODO: configurable?
n_hidden = 256 # TODO: configurable?
nc = 40

In [4]:
def make_model(batch_size_override=None):
    if batch_size_override is None:
        batch_size_override = bs
    model = Sequential([
        Embedding(vocab_size, n_fac, input_length=nc, batch_input_shape=(batch_size_override,nc)),
        BatchNormalization(),
        LSTM(n_hidden, input_dim=n_fac, return_sequences=True, stateful=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        LSTM(n_hidden, input_dim=n_fac, return_sequences=True, stateful=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam())
    return model

In [5]:
model = make_model(batch_size_override=1)

In [6]:
saved_path = os.path.join(model_path, 'shakespeare_2.h5')
model.load_weights(saved_path)

In [7]:
import numpy as np
import re

In [8]:
def get_valid_line(line):
    regex = re.compile(r"<([^>/]+)>([^<]+)<\/([^>]+)>")
    if regex.match(line) != None:
        valid_line = re.sub(regex, r"<\1>\2</\1>", line)
        return valid_line

In [9]:
def fix_newlines(text):
    return "\n".join(text.splitlines())

In [10]:
def generate_conversation(m, seed, num_lines=5):
    binary_tags = ['Brock', 'Ash']; # TODO: configurable
    curr_tag_idx = 0;
    
    i = 0    c 
    output = ['']*num_lines

    output[i] = seed
    valid_line = get_valid_line(output[i])
    if valid_line != None:
        output[i] = valid_line
        i += 1
        if (i < num_lines):
            curr_tag_idx = abs(curr_tag_idx - 1)
            output[i] = f'<{binary_tags[curr_tag_idx]}>'
    
    while i < num_lines:
        x = np.array([char_indices[c] for c in ''.join(output)[-nc:]])[np.newaxis,:]
        preds = m.predict(x, verbose=0, batch_size=1)[0][-1]
        preds = preds / np.sum(preds)
        new_char = np.random.choice(chars, p=preds)
        output[i] += new_char
        valid_line = get_valid_line(output[i])
        if valid_line != None:
            output[i] = valid_line
            i += 1
            if (i < num_lines):
                curr_tag_idx = abs(curr_tag_idx - 1)
                output[i] = f'<{binary_tags[curr_tag_idx]}>'
    print(fix_newlines('\n'.join(output)))

In [11]:
generate_conversation(model, '<Brock>Courteous lord, one word.</Brock>', 10)

<Brock>Courteous lord, one word.</Brock>
<Ash>Ah, my bring us.
</Ash>
<Brock>Sir, you nothing k Dropposh>As your perficients, thou art like a friend,
    For threat me a kind lies one you less.
</Brock>
<Ash>Fie, fair them, and your desert may do pity beg?
    I were a borrow for rich passion to hard to thee!
    I would friar.
</Ash>
<Brock>From th' bent, and so, or perform'd,
    But I am so iner of water, being weaked, sir.   For that they will our dismission; but when a rather
    be book'd him: the love. Farewell, it is a gossess, yet from brows: wherefore affect
    By no more in my blood
    Your another humour with them, welcome, for God!
</Brock>
<Ash>But I am wagger one as lost to me, by the youth' beg will be banish'd
    Look to show so hiever.
    The digs seems him to he on super? Thou'dst
    have tell you whom, from which you, should have madness himself
    Was gallows his beard which to still to mine untimes,
    Soon and too much apon him? Thy sconclusion wrongs, we 